<br>

# Introdução

Publicado em 2019, ano de referência de 2018

In [ ]:
#!pip3 install traquitanas --upgrade

In [ ]:
import shutil
import urllib.parse
import urllib.request
from pathlib import Path
from zipfile import ZipFile

import pandas as pd
from paths import data_path, input_path_temp, output_path
from traquitanas.files import utils

<br>

# Functions


<br>

## _Download_


In [ ]:
url = 'https://www.gov.br/mdr/pt-br/assuntos/saneamento/snis/diagnosticos-anteriores-do-snis/agua-e-esgotos-1/2018/Planilhas_AE2018.zip'
zipfile_filepath = input_path_temp / Path(url).name
zipfile_filepath


In [ ]:
# Download the file from 'url' and save it locally under 'filename'
with urllib.request.urlopen(url) as r, open(zipfile_filepath, 'wb') as out_file:
    shutil.copyfileobj(r, out_file)


<br>

## _Unzip_


In [ ]:
files = input_path_temp.rglob('*.zip')
for file in files:
    print(file)
    with ZipFile(file, 'r') as zip_obj:
        for names in zip_obj.namelist():
            zip_obj.extract(names, input_path_temp)
        zip_obj.close()

files = input_path_temp.rglob('*.zip')
for file in files:
    print(file)
    with ZipFile(file, 'r') as zip_obj:
        for names in zip_obj.namelist():
            zip_obj.extract(names, input_path_temp)
        zip_obj.close()


<br>

## Tabela de Indicadores

Lê e Ajusta


In [ ]:
list_type = list(input_path_temp.rglob('*.xls'))
list_type = [x for x in list_type if 'Indicadores' in str(x)]
list_type[0:10]


In [ ]:
dfs = []

for file in list_type:
    print(file)
    df = pd.read_excel(file, header=[0, 1, 2, 3], skiprows=range(0, 6))

    # Ajusta o Primeiro Nível do Cabeçalho
    a = df.columns.get_level_values(0)
    a = a.to_series()
    a = a.mask(lambda x: x.str.startswith('Unnamed')).ffill()
    a = a.replace('Tipo de serviço', 'PRESTADOR DE SERVIÇOS')
    a = a.str.title()
    a = a.replace(
        {'  ': ' ', ' E ': ' e ', ' De ': ' de ', ' Sobre ': ' sobre '},
        regex=True,
    )
    a = a.to_list()

    # Ajusta o Segundo Nível do Cabeçalho
    b = df.columns.get_level_values(1)
    b = b.to_series()
    b = b.mask(lambda x: x.str.startswith('Unnamed'))
    b = b.fillna('Tipo de serviço')
    b = b.to_list()

    # Ajusta o Terceiro Nível do Cabeçalho
    c = df.columns.get_level_values(2)
    c = c.to_series()
    c = c.replace('-', '')
    c = '(' + c + ')'
    c = c.replace('()', '')
    c = c.to_list()

    # Ajusta o Quarto Nível do Cabeçalho
    d = df.columns.get_level_values(3)
    d = d.to_series()
    d = d.replace('-', '')
    d = '(' + d + ')'
    d = d.replace('()', '')
    d = d.to_list()

    # Junta tudo
    z = [x[0] + ' ' + x[1] + ' ' + x[2] for x in zip(b, c, d)]
    z = pd.Series(z)
    z = z.str.strip()
    z.to_list()

    # Define as Colunas
    # df.columns = [a, z]
    df.columns = z

    dfs.append(df)

dfs = pd.concat(dfs)
df = dfs


In [ ]:
df = df[df['UF'] == 'SP']

df.to_csv(
    output_path / 'tab_2018_indicadores_SNIS.csv',
    index=False,
)
df.to_excel(
    output_path / 'tab_2018_indicadores_SNIS.xlsx',
    sheet_name='SNIS 2018 Indicadores',
    index=False,
)


df.info()
df.head()


<br>

## Tabela de Informações

Lê e Ajusta


In [ ]:
list_type = list(input_path_temp.rglob('*.xls'))
list_type = [x for x in list_type if 'Informacoes' in str(x)]
list_type[0:10]


In [ ]:
dfs = []

for file in list_type:
    df = pd.read_excel(file, header=[0, 1, 2, 3, 4, 5], skiprows=range(0, 6))

    # Ajusta o Primeiro Nível do Cabeçalho
    a = df.columns.get_level_values(0)
    a = a.to_series()
    a = a.mask(lambda x: x.str.startswith('Unnamed')).ffill()
    a = a.str.title()
    a = a.replace(
        {
            '  ': ' ',
            ' E ': ' e ',
            ' De ': ' de ',
            ' Com ': ' com ',
            ' Sobre ': ' sobre ',
        },
        regex=True,
    )
    a = a.str.strip()
    a = a.to_list()

    # Ajusta o Segundo Nível do Cabeçalho
    b = df.columns.get_level_values(1)
    b = b.to_series()
    b = b.mask(lambda x: x.str.startswith('Unnamed')).ffill()
    b = b.str.title()
    b = b.replace(
        {
            ' À ': ' à ',
            ' Ao ': ' ao ',
            ' Ao ': ' ao ',
            ' Da ': ' da ',
            ' Das ': ' das ',
            ' De ': ' de ',
            ' Do ': ' do ',
            ' Dos ': ' dos ',
            ' Com ': ' com ',
            ' A ': ' a ',
            ' As ': ' as ',
            ' E ': ' e ',
            ' O ': ' o ',
            ' Os ': ' os ',
            ' Na ': ' na ',
            ' No ': ' no ',
            ' Nas ': ' nas ',
            ' Não ': ' não ',
            ' Em ': ' em ',
            ' Pelo ': ' pelo ',
            ' Para ': ' para ',
            ' Qual ': ' qual ',
            ' Por ': ' por ',
            ' Se ': ' se ',
            ' Ou ': ' ou ',
            '–': '-',
        },
        regex=True,
    )
    b = b.str.strip()
    b = b.replace('Uf', 'UF')
    b = b.fillna('')
    b = b.to_list()

    # Ajusta o Terceiro Nível do Cabeçalho
    c = df.columns.get_level_values(2)
    c = c.to_series()
    c = c.mask(lambda x: x.str.startswith('Unnamed'))
    c = c.fillna('')
    c = c.str.title()
    c = c.replace(
        {
            ' À ': ' à ',
            ' Ao ': ' ao ',
            ' Ao ': ' ao ',
            ' Da ': ' da ',
            ' Das ': ' das ',
            ' De ': ' de ',
            ' Do ': ' do ',
            ' Dos ': ' dos ',
            ' Com ': ' com ',
            ' A ': ' a ',
            ' E ': ' e ',
            ' O ': ' o ',
            ' Os ': ' os ',
            ' Na ': ' na ',
            ' No ': ' no ',
            ' Nas ': ' nas ',
            ' Não ': ' não ',
            ' Em ': ' em ',
            ' Pelo ': ' pelo ',
            ' Para ': ' para ',
            ' Qual ': ' qual ',
            ' Por ': ' por ',
            ' Se ': ' se ',
            ' Ou ': ' ou ',
        },
        regex=True,
    )
    c = c.replace('-', '')
    c = c.replace('()', '')
    c = c.str.strip()
    c = c.to_list()

    # Ajusta o Quarto Nível do Cabeçalho
    d = df.columns.get_level_values(3)
    d = d.to_series()
    d = d.mask(lambda x: x.str.startswith('Unnamed'))
    d = d.fillna('')
    d = d.str.title()
    d = d.replace(
        {
            ' À ': ' à ',
            ' Ao ': ' ao ',
            ' Ao ': ' ao ',
            ' Da ': ' da ',
            ' Das ': ' das ',
            ' De ': ' de ',
            ' Do ': ' do ',
            ' Dos ': ' dos ',
            ' Com ': ' com ',
            ' A ': ' a ',
            ' As ': ' as ',
            ' E ': ' e ',
            ' O ': ' o ',
            ' Na ': ' na ',
            ' No ': ' no ',
            ' Nas ': ' nas ',
            ' Não ': ' não ',
            ' Em ': ' em ',
            ' Pelo ': ' pelo ',
            ' Para ': ' para ',
            ' Qual ': ' qual ',
            ' Por ': ' por ',
            ' Se ': ' se ',
            ' Ou ': ' ou ',
        },
        regex=True,
    )
    d = d.replace('()', '')
    d = d.to_list()

    # Ajusta o Quinto Nível do Cabeçalho
    e = df.columns.get_level_values(4)
    e = e.to_series()
    e = e.mask(lambda x: x.str.startswith('Unnamed'))
    e = e.fillna('')
    e = e.replace('-', '')
    e = '(' + e + ')'
    e = e.replace('()', '')
    e = e.replace(' / ', '/', regex=True)
    e = e.to_list()

    # Ajusta o Sexto Nível do Cabeçalho
    f = df.columns.get_level_values(5)
    f = f.to_series()
    f = f.mask(lambda x: x.str.startswith('Unnamed'))
    f = f.fillna('')
    f = f.replace('-', '')
    f = '(' + f + ')'
    f = f.replace('()', '')
    f = f.to_list()

    # Junta tudo
    z = [
        x[0] + ' ' + x[1] + ' ' + x[2] + ' ' + x[3] + ' ' + x[4]
        for x in zip(b, c, d, e, f)
    ]
    z = pd.Series(z)
    z = z.replace({'/  /': '/', '  ': ' '}, regex=True)
    z = z.str.strip()
    z.to_list()

    # Define as Colunas
    # df.columns = [a, z]
    df.columns = z

    dfs.append(df)

dfs = pd.concat(dfs)
df = dfs


In [ ]:
df = df[df['UF'] == 'SP']

df.to_csv(
    output_path / 'tab_2018_informações_SNIS.csv',
    index=False,
)
df.to_excel(
    output_path / 'tab_2018_informações_SNIS.xlsx',
    sheet_name='SNIS 2018 Informações',
    index=False,
)

df.info()
df.head()


In [ ]:
shutil.rmtree(
    input_path_temp,
    ignore_errors=False,
    onerror=utils.handle_remove_readonly
)